In [56]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent))
str(Path.cwd().parent)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'c:\\Users\\Joaquín Amat\\Documents\\GitHub\\skforecast'

In [57]:
# Libraries
# ==============================================================================
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor

from skforecast.datasets import fetch_dataset
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries import grid_search_forecaster_multiseries
from skforecast.model_selection_multiseries import bayesian_search_forecaster_multiseries

In [58]:
# Data download
# ==============================================================================
data = fetch_dataset(name="items_sales")
data.head()

items_sales
-----------
Simulated time series for the sales of 3 different items.
Simulated data.
Shape of the dataset: (1097, 3)


,item_1,item_2,item_3
date,,,
2012-01-01,8.253175,21.047727,19.429739
2012-01-02,22.777826,26.578125,28.009863
2012-01-03,27.549099,31.751042,32.078922
2012-01-04,25.895533,24.567708,27.252276
2012-01-05,21.379238,18.191667,20.357737


In [59]:
# Split data into train-val-test
# ==============================================================================
end_train = '2014-07-15 23:59:00'
data_train = data.loc[:end_train, :].copy()
data_test  = data.loc[end_train:, :].copy()

print(
    f"Train dates : {data_train.index.min()} --- {data_train.index.max()}   "
    f"(n={len(data_train)})"
)
print(
    f"Test dates  : {data_test.index.min()} --- {data_test.index.max()}   "
    f"(n={len(data_test)})"
)

Train dates : 2012-01-01 00:00:00 --- 2014-07-15 00:00:00   (n=927)
Test dates  : 2014-07-16 00:00:00 --- 2015-01-01 00:00:00   (n=170)


In [71]:
# Create and fit a Forecaster Multi-Series
# ==============================================================================
forecaster = ForecasterAutoregMultiSeries(
                 regressor          = LGBMRegressor(random_state=123),
                 lags               = 3,
                 encoding           = 'ordinal',
                 transformer_series = None,
                 transformer_exog   = None,
                 weight_func        = None,
                 series_weights     = None,
                 differentiation    = None,
                 dropna_from_series = False,
                 fit_kwargs         = None,
                 forecaster_id      = None
             )

forecaster.fit(series=data_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000070 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 768
[LightGBM] [Info] Number of data points in the train set: 2772, number of used features: 4
[LightGBM] [Info] Start training from score 18.713730


In [67]:
X_train = forecaster.create_train_X_y(series=data_train)[0]
X_train

,lag_1,lag_2,lag_3,_level_skforecast
date,,,,
2012-01-04,27.549099,22.777826,8.253175,0
2012-01-05,25.895533,27.549099,22.777826,0
2012-01-06,21.379238,25.895533,27.549099,0
2012-01-07,21.106643,21.379238,25.895533,0
2012-01-08,20.533871,21.106643,21.379238,0
...,...,...,...,...
2014-07-11,9.677730,12.199832,14.072343,2
2014-07-12,10.396751,9.677730,12.199832,2
2014-07-13,16.139173,10.396751,9.677730,2


In [68]:
lw = forecaster.last_window['item_1'] + 10
lw = lw.to_frame(name='item_4')
lw

,item_4
date,
2014-07-13,32.609388
2014-07-14,33.307307
2014-07-15,35.980745


In [69]:
forecaster.predict(steps=1, levels='item_4', last_window=lw)

c:\Users\Joaquín Amat\Documents\GitHub\skforecast\skforecast\utils\utils.py:738: UserWarning: `levels` {'item_4'} were not included in training. Unknown levels are theatened as NaN, which may cause the prediction to fail if the regressor does not accept NaN values.
  warnings.warn(


,item_4
2014-07-16,29.320733


In [70]:
X_predict = forecaster.create_predict_X(steps=1, levels='item_4', last_window=lw)['item_4']
X_predict

c:\Users\Joaquín Amat\Documents\GitHub\skforecast\skforecast\utils\utils.py:738: UserWarning: `levels` {'item_4'} were not included in training. Unknown levels are theatened as NaN, which may cause the prediction to fail if the regressor does not accept NaN values.
  warnings.warn(
c:\Users\Joaquín Amat\Documents\GitHub\skforecast\skforecast\utils\utils.py:738: UserWarning: `levels` {'item_4'} were not included in training. Unknown levels are theatened as NaN, which may cause the prediction to fail if the regressor does not accept NaN values.
  warnings.warn(


,lag_1,lag_2,lag_3,_level_skforecast
2014-07-16,35.980745,33.307307,32.609388,NaN


In [48]:
forecaster.regressor.predict(X_predict)

array([29.60930223])

In [ ]:
X_predict['_level_skforecast'] = np.nan
X_predict

,lag_1,lag_2,lag_3,_level_skforecast
2014-07-16,10.489583,10.895833,8.1,NaN


In [ ]:
forecaster.regressor.predict(X_predict)

array([10.6616944])